## Qual o tempo de uso das bombas em horário de ponta e fora de ponta?

In [1]:
from pathlib import Path
import pandas as pd 

In [2]:
df_path = Path("../data/curated_data/water_consumption_curated_1.partquet")
df = pd.read_parquet(df_path)
df.head()

,timestamp,flow_in_(l/s),reservoir_level_(%),pressure_(mca),gmb_1_is_on,gmb_2_is_on,reservoir_level_liters,time_passed_seconds,liters_entered,flow_out_(l/s)
0,2023-03-17 11:27:06,68.59,29.86,38.20,0,1,298600.0,NaN,NaN,NaN
1,2023-03-17 12:28:56,66.05,35.86,38.20,0,1,358600.0,3710.0,60000.0,49.877493
2,2023-03-17 12:31:26,65.64,36.16,38.06,0,1,361600.0,150.0,3000.0,45.640000
3,2023-03-17 12:33:56,65.64,36.50,38.03,0,1,365000.0,150.0,3400.0,42.973333
4,2023-03-17 12:36:26,65.64,36.80,38.17,0,1,368000.0,150.0,3000.0,45.640000


In [3]:
# Converter a coluna 'timestamp' para datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Adicionar uma coluna para a hora do dia e o dia
df['hour'] = df['timestamp'].dt.hour
df['day'] = df['timestamp'].dt.date

# Definir horários de ponta (18:00 - 21:00)
peak_hours = (df['hour'] >= 18) & (df['hour'] <= 21)
df['is_peak'] = peak_hours

# Filtrar os dados que não têm valores nulos na coluna 'time_passed_seconds'
df = df.dropna(subset=['time_passed_seconds'])

# Calcular o tempo total de uso das bombas em horário de ponta e fora de ponta
df['gmb_1_duration'] = df['gmb_1_is_on'] * df['time_passed_seconds']
df['gmb_2_duration'] = df['gmb_2_is_on'] * df['time_passed_seconds']

# Agrupar por dia e por horário de ponta, somar o tempo de uso das bombas
daily_peak_usage = df[df['is_peak']].groupby('day').agg({'gmb_1_duration': 'sum', 'gmb_2_duration': 'sum'})
daily_off_peak_usage = df[~df['is_peak']].groupby('day').agg({'gmb_1_duration': 'sum', 'gmb_2_duration': 'sum'})

# Calcular a média diária do tempo de uso das bombas em horários de ponta e fora de ponta
gmb_1_peak_avg = daily_peak_usage['gmb_1_duration'].mean() / 3600  # convertendo para horas
gmb_1_off_peak_avg = daily_off_peak_usage['gmb_1_duration'].mean() / 3600  # convertendo para horas

gmb_2_peak_avg = daily_peak_usage['gmb_2_duration'].mean() / 3600  # convertendo para horas
gmb_2_off_peak_avg = daily_off_peak_usage['gmb_2_duration'].mean() / 3600  # convertendo para horas

print(f"Tempo médio de uso da bomba 1 em horário de ponta: {gmb_1_peak_avg:.2f} horas")
print(f"Tempo médio de uso da bomba 1 em horário fora de ponta: {gmb_1_off_peak_avg:.2f} horas")

print(f"Tempo médio de uso da bomba 2 em horário de ponta: {gmb_2_peak_avg:.2f} horas")
print(f"Tempo médio de uso da bomba 2 em horário fora de ponta: {gmb_2_off_peak_avg:.2f} horas")

Tempo médio de uso da bomba 1 em horário de ponta: 1.41 horas
Tempo médio de uso da bomba 1 em horário fora de ponta: 8.00 horas
Tempo médio de uso da bomba 2 em horário de ponta: 0.76 horas
Tempo médio de uso da bomba 2 em horário fora de ponta: 4.44 horas
